# SQLAlchemy

Here are the [docs](https://docs.sqlalchemy.org/en/14/tutorial/)

In [ ]:
! pip install psycopg2
! pip install SQLAlchemy

In [ ]:
import json
import logging


with open("credentials.json", "r") as f:
    creds = json.load(f)

In [ ]:
from sqlalchemy import create_engine
# engine = create_engine("sqlite+pysqlite:///:memory:", echo=True, future=True)
postgres_db_url = f"postgresql+psycopg2://{creds['username']}:{creds['password']}@{creds['host']}/{creds['default-database']}"
engine = create_engine(postgres_db_url)


In [ ]:
from sqlalchemy import text

with engine.connect() as conn:
    result = conn.execute(text("select 'hello world'"))
    print(result.all())

## Creating a table

In [ ]:
with engine.connect() as conn:
    # My DB needs a bit of work done to it!
    # conn.execute(text("DROP TABLE some_table"))
    conn.execute(text("CREATE TABLE some_table (x int, y int)"))
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 1, "y": 1}, {"x": 2, "y": 4}]
    )
    # conn.commit()

In [ ]:
with engine.connect() as conn:
    # See https://docs.sqlalchemy.org/en/14/core/connections.html#sqlalchemy.engine.ResultProxy
    # for more information
    results = conn.execute(text("SELECT * FROM some_table"))
    print(results.first())

# ORM - Object Relational Mapping

In [ ]:
from sqlalchemy.orm import registry
from sqlalchemy import MetaData
metadata_obj = MetaData()
metadata_obj.clear()
mapper_registry = registry()
Base = mapper_registry.generate_base()

In [ ]:
from sqlalchemy.orm import relationship
from sqlalchemy import ForeignKey


class User(Base): # see https://docs.sqlalchemy.org/en/14/tutorial/metadata.html#tutorial-orm-table-metadata for more info
    __tablename__ = 'user_account'
    id = Column(Integer, primary_key=True)
    name = Column(String(30))
    fullname = Column(String)

    addresses = relationship("Address", back_populates="user")

    def __repr__(self):
        return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"

class Address(Base):
    __tablename__ = 'address'

    id = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey('user_account.id'))
    user = relationship("User", back_populates="addresses")

    def __repr__(self):
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"

In [ ]:
# Create the tables!
Base.metadata.create_all(engine)

In [ ]:
user_table.c.name

In [ ]:
user_table.c.keys()

## Inserting data

In [ ]:
from sqlalchemy import insert
stmt = insert(user_table).values(name='spongebob', fullname="Spongebob Squarepants")
# What is stmt?
print(stmt)

In [ ]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind = engine)
session = Session() # See this link for more information https://www.tutorialspoint.com/sqlalchemy/sqlalchemy_orm_adding_objects.htm

with engine.connect() as conn:
    result = conn.execute(stmt)
    session.commit()

In [ ]:
with engine.connect() as conn:
    # See https://docs.sqlalchemy.org/en/14/core/connections.html#sqlalchemy.engine.ResultProxy
    # for more information
    results = conn.execute(text("SELECT * FROM user_account"))
    print(results.first())

In [ ]:
from sqlalchemy import select
stmt = select(user_table)
print(stmt)

In [ ]:
with engine.connect() as conn:
    # See https://docs.sqlalchemy.org/en/14/core/connections.html#sqlalchemy.engine.ResultProxy
    # for more information
    results = conn.execute(stmt)
    print(results.first())

In [ ]:
from sqlalchemy import select
stmt = select(user_table).where(user_table.c.name == 'spongebob')
print(stmt)

In [ ]:
with engine.connect() as conn:
    for row in conn.execute(stmt):
        print(row)

## Cleanup tables

In [ ]:
from sqlalchemy import select

Base.metadata.drop_all(engine)